# Imports

In [1]:
#  Load the "autoreload" extension so that code can change
%load_ext autoreload
%reload_ext autoreload
from pathlib import Path

#  always reload modules so that as you change code in src, it gets loaded
%autoreload 2
%matplotlib inline

import sys
sys.path.append('../')
from src.imports import *
from src.data.download_data import *
from src.data.fire_data import *
from src.data.read_data import *
from src.gen_functions import *
from src.features.build_features import *
from src.visualization.visualize import *
import seaborn as sns
output_notebook()
# set font size 
plt.rcParams.update({'font.size': 16})

from datetime import timedelta
from matplotlib.ticker import MaxNLocator# transition from acceptable to unhealthy for sensitive group and to unhealthy. 


Loading BokehJS ...

In [2]:
# transition from acceptable to unhealthy for sensitive group and to unhealthy. 
transition_dict = { 'PM2.5': [0, 35.5, 55.4, 1e3],
                  'PM10': [0, 155, 254, 1e3],
                  'O3':[0, 70 , 85, 1e3],
                  'SO2':[0, 75, 185, 1e3],
                  'NO2': [0, 100, 360, 1e3],
                  'CO': [0, 6.4, 12.5, 1e3]}

gas_list = ['PM2.5', 'PM10', 'O3', 'CO', 'NO2', 'SO2']

city_info = {'Country': 'Thailand',
 'City': 'Chiang Mai',
 'City (ASCII)': 'Chiang Mai',
 'Region': 'Chiang Mai',
 'Region (ASCII)': 'Chiang Mai',
 'Population': '200952',
 'Latitude': '18.7904',
 'Longitude': '98.9847',
 'Time Zone': 'Asia/Bangkok'}

city_name = city_info['City'].lower().replace(' ', '_')

x = merc_x(city_info['Longitude'])
y = merc_y(city_info['Latitude'])

In [3]:
b_folder='../data/pm25/'
a4th_folder ='../data/air4thai_hourly/'
cm_folder ='../data/cm_proc/'
cdc_folder = '../data/cdc_data/'
aqm_folder = '../data/aqm_hourly2/'
model_folder = f'../models/{city_name}/'
if not os.path.exists(model_folder):
    os.mkdir(model_folder)

In [4]:
aqm1 = pd.read_csv(cm_folder + '35t.csv').set_index('datetime').dropna(how='all')
aqm1.index = pd.to_datetime(aqm1.index)
aqm2 = pd.read_csv(cm_folder + '36t.csv').set_index('datetime').dropna(how='all')
aqm2.index = pd.to_datetime(aqm2.index)
print(aqm2.columns)
# keep only the data after the satallite data which is 200-11-11 13 am
aqm2_01 = aqm2[aqm2.index>='2000-11-01 00:00:00'].copy()
aqm2_01 = add_season(aqm2_01)
print(aqm2_01.shape)

# weather data 
filename = 'C:/Users/Benny/Documents/Fern/aqi_thailand2/data/weather_cities/Mueang_Chiang_Mai.csv'
wea = pd.read_csv(filename)
wea.drop(['Time','Dew Point(C)','Wind Gust(kmph)','Pressure(in)','Precip.(in)'], axis=1, inplace=True)
wea['datetime'] = pd.to_datetime(wea['datetime'])
# merge with weather 

aqm2_01 = aqm2_01.merge(wea, left_index=True, right_on ='datetime',how='inner').set_index('datetime')
print(aqm2_01.shape)

Index(['CO', 'O3', 'NO2', 'SO2', 'PM10', 'PM2.5'], dtype='object')
(168337, 8)
(165459, 13)


In [5]:
fire = pd.read_csv('C:/Users/Benny/Documents/Fern/aqi_thailand2/data/cm_proc/file_m_proc.csv')
fire['datetime'] = pd.to_datetime(fire['datetime'])
fire = fire.set_index('datetime')
fire.head()

,confidence,lat_km,long_km,distance,power,count
datetime,,,,,,
2002-07-04 13:03:00,67,1612.0,11453.0,665.551109,120.96,1
2002-07-04 13:04:00,52,2246.0,11701.0,694.254989,20.52,1
2002-07-04 13:04:00,48,2304.0,11682.0,689.069015,13.50,1
2002-07-04 13:04:00,91,2305.0,11682.0,689.341767,132.30,1
2002-07-04 13:05:00,62,2428.0,11936.0,968.527252,10.45,1


In [6]:
# function for feature eng fire
def cal_power_damp(series: pd.core.series.Series, distance: pd.core.series.Series, surface='sphere'):
    """ Calculate the damped power based on the distance series. 

    The damping factor maybe 1/distance or 1/distance**2.
    Args: 
        series: series to recalculate
        distance: distance array. Must have the same lenght as the series
        surface(optional): either 'circle' or 'sphere'

    Returns:
        new_series

    Examples:
        cal_power_damp(fire['power'], fire['distance'],surface='sphere')

    """
    if surface == 'sphere':
        new_series = series/distance**2

    elif surface == 'circle':
        new_series = series/distance

    return new_series


def cal_arrival_time(detection_time: pd.core.series.Series, distance: pd.core.series.Series, wind_speed: (float, np.array) = 2):
    """ Calculate the approximate time that the pollution arrived at the city using the wind speed and distance from the hotspot.

    Round arrival time to hour 

    Args:
        detection_time: datetime series
        distance: distance series in km
        wind_speed(optional): approximate wind speed, can be floar or array in km/hour

    Returns: 
        arrival_time: datetime series of arrival time

    """
    arrival_time = detection_time + pd.to_timedelta(distance/wind_speed, 'h')
    return arrival_time.dt.round('H')


def shift_fire(fire_df: pd.core.frame.DataFrame, fire_col: str = 'power', damp_surface: str = 'sphere', shift: int = 0, roll: int = 48, w_speed: (float, int) = 8):
    """ Feature engineer fire data. Account of the distance from the source and time lag using wind speed.

    Args:
        fire_df:
        fire_col
        damp_surface
        shift
        roll

    """
    require_cols = ['distance', fire_col]
    if fire_df.columns.isin(require_cols).sum() > len(require_cols):
        raise AssertionError(
            'missing required columns for feature engineering fire data')

    # calculate the damping factors
    fire_df['damp_'+fire_col] = cal_power_damp(
        fire_df[fire_col], fire_df['distance'], surface=damp_surface)
    # calculate particle arrival time
    fire_df['arrival_time'] = cal_arrival_time(
        detection_time=fire_df.index, distance=fire_df['distance'], wind_speed=w_speed)
    fire_df = fire_df.set_index('arrival_time')
    fire_df = fire_df.resample('h').sum()['damp_'+fire_col]
    fire_df = fire_df.rolling(roll).sum()
    fire_df = fire_df.shift(shift)
    fire_df.index.name = 'datetime'
    return fire_df


def get_fire_feature(fire, zone_list=[0, 100, 200, 400, 800, 1000], fire_col: str = 'power', damp_surface: str = 'sphere', shift: int = 0, roll: int = 48, w_speed: (float, int) = 8):
    """ Separate fire from different distance

    """
    fire_col_list = []
    new_fire = pd.DataFrame()
    for start, stop in zip(zone_list, zone_list[1:]):
        col_name = f'fire_{start}_{stop}'

        fire_col_list.append(col_name)
        # select sub-data baseline the distance
        fire_s = fire[(fire['distance'] < stop) & (fire['distance'] >= start)][[fire_col, 'distance']].copy()
        fire_s = shift_fire(fire_s, fire_col=fire_col, damp_surface=damp_surface,
                            shift=shift, roll=roll, w_speed=w_speed)
        fire_s.name = col_name
        new_fire = pd.concat([new_fire, fire_s], axis=1, ignore_index=False)

    new_fire = new_fire.fillna(0)
    return new_fire, fire_col_list

def sep_fire_zone(fire, fire_col, zone_list=[0, 100, 200, 400, 800, 1000]):
    """ Separate fire data into zone mark by a distance in the zone_list without perform feature enginering.
    Use for data visualization
    
    Args: 
        fire: fire dataframe
        fire_col: 'power' or 'count'
        zone_list:
        
    Return: 
        new_fire: a dataframe with each column, a fire data in that zone
        fire_col_list: a list of column name
    
    """
    fire_col_list = []
    new_fire = pd.DataFrame()
    for start, stop in zip(zone_list, zone_list[1:]):
        col_name = f'fire_{start}_{stop}'
        fire_col_list.append(col_name)
        # select sub-data baseline the distance
        fire_s = fire[(fire['distance'] < stop) & (fire['distance'] >= start)][[fire_col]].copy()
        fire_s.columns = [col_name]
        fire_s = fire_s.resample('h').sum()
        new_fire = pd.concat([new_fire, fire_s], axis=1, ignore_index=False)

    return new_fire, fire_col_list

In [7]:
fire_dict = {'fire_col': 'power',
 'surface': 'sphere',
 'w_speed': 10,
 'shift': -24,
 'roll': 72}

In [8]:
def split_data_index(data_index, shuffle=False,val_size=0.3, test_size=0.25):
    trn, test = train_test_split(data_index, test_size=test_size, shuffle=False)
    trn, val  = train_test_split(trn, test_size=val_size, shuffle=False)
    print('train size', len(trn))
    print('validation size',len(val))
    print('test size',len(test))
    return trn,  val, test

def get_data_matrix(data, pollutant,use_index, x_cols=[]):
    """Extract data in data dataframe into x,y matricies using use_index.
    
    """
    temp = data.loc[use_index]
    
    y = temp[pollutant].values
    
    if len(x_cols)==0:
        x = temp.drop(pollutant,axis=1)
    else:
        x = temp[x_cols]
        
    x_cols = x.columns
    return x.values, y, x_cols

# Add AutoRegressive Feature

# calculate number of lag 
def find_num_lag(poll_series, thres=0.5):
    """ Calculate the numbers of partial autocorrelation lag to add as feature to a time series. 
    
    """

    pac = pacf(poll_series)
    # find the number of lag 
    idxs = np.where(pac >= 0.5)[0]
    return idxs[1:]

def add_lags(data, pollutant):
    """Add lags columns to x_data.

    """
    # calculate num lags
    num_lags = find_num_lag(data[pollutant])
    for idx in num_lags:
        lag_name = f'{pollutant}_lag_{idx}'
        lag_series = data[pollutant].shift(idx) 
        lag_series.name = lag_name
        # add to data 
        data = pd.concat([data, lag_series], axis=1) 
    data = data.dropna()
    return data

In [9]:
# load the data and keep only relavant data 
pollutant = 'PM2.5'
cols = [pollutant, 'Temperature(C)', 'Humidity(%)', 'Wind', 'Wind Speed(kmph)', 'Condition']
data = aqm2_01[cols].dropna()

if pollutant == 'PM2.5':
    data = data.loc['2010':]

# add weather 
dummies = wind_to_dummies(data['Wind'])
data.drop('Wind',axis=1, inplace=True)
data = pd.concat([data, dummies], axis=1)
data = add_is_rain(data)
data = add_calendar_info(data)
data = add_lags(data, pollutant)
data_no_fire = data.astype(float)


In [10]:
# split data index 
# because will be building new feature later in the pipeline, therefore only need index here.
trn_idx, val_idx, test_idx = split_data_index(data.index,shuffle=False, val_size=0.4, test_size=0.3)
# split furture for rf 
trn_rf_idx, val_rf_idx = train_test_split(trn_idx,test_size=0.35)

train size 31401
validation size 20935
test size 22431


In [11]:
# implement the best parameters 
fire_col = fire_dict['fire_col']
w_speed = fire_dict['w_speed']
damp_surface = fire_dict['surface']
shift = fire_dict['shift']
roll = fire_dict['roll']
# obtain fire data 
fire_proc, fire_col_list = get_fire_feature(fire, zone_list=[0, 100, 200, 400, 800, 1000], 
fire_col=fire_col,damp_surface=damp_surface, 
shift=shift, roll=roll, w_speed=w_speed)
 
# merge fire data 
data = data_no_fire.merge(fire_proc, left_index=True, right_index=True, how='inner')
data = data.dropna()
 
x_cols = ['Wind Speed(kmph)', 'Temperature(C)', 'Humidity(%)', 'time_of_day',
       'fire_400_800', 'fire_800_1000', 'fire_200_400', 'fire_0_100',
       'fire_100_200', 'PM2.5_lag_1']
xtrn, ytrn, x_cols = get_data_matrix(data, pollutant,trn_idx, x_cols=x_cols)
xval, yval, _ = get_data_matrix(data, pollutant,val_idx, x_cols=x_cols)

In [12]:
#ask TPOT to hunt for the best model
tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2)
tpot.fit(xtrn, ytrn)


Generation 1 - Current best internal CV score: -118.70543593428506
Generation 2 - Current best internal CV score: -118.70543593428506
Generation 3 - Current best internal CV score: -118.70543593428506
Generation 4 - Current best internal CV score: -118.70543593428506
Generation 5 - Current best internal CV score: -118.70543593428506
Best pipeline: XGBRegressor(AdaBoostRegressor(input_matrix, learning_rate=0.01, loss=linear, n_estimators=100), learning_rate=0.1, max_depth=3, min_child_weight=13, n_estimators=100, nthread=1, objective=reg:squarederror, subsample=0.7000000000000001)


TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
              disable_update_check=False, early_stop=None, generations=5,
              log_file=<ipykernel.iostream.OutStream object at 0x00000189E2DF4088>,
              max_eval_time_mins=5, max_time_mins=None, memory=None,
              mutation_rate=0.9, n_jobs=1, offspring_size=None,
              periodic_checkpoint_folder=None, population_size=50,
              random_state=None, scoring=None, subsample=1.0, template=None,
              use_dask=False, verbosity=2, warm_start=False)

In [14]:
print(tpot.score(xval,yval))
tpot.export('tpot_pipeline.py')

-62.410713640603696
